In [6]:
import pandas as pd
ratings_movies=pd.read_csv('data/ratings_movies.csv', sep=',')
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

# Задание 1.
# Создайте в таблице новый признак year_release, который соответствует году выпуска фильма. 
# У скольких фильмов не указан год их выпуска?
ratings_movies['year_release']=ratings_movies['title'].apply(get_year_release)
ratings_movies.info()

# Задание 2. Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей? 
# В качестве ответа запишите название этого фильма без указания года его выпуска.
mask1=ratings_movies['year_release']==1999 #обозначил маску на 1999
ratings_movies[mask1].groupby('title')['rating'].mean().sort_values()

# Задание 3.
# Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
# Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).
mask2=ratings_movies['year_release']==2010
ratings_movies[mask2].groupby('genres')['rating'].mean().sort_values()

# Задание 4.
# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) 
# фильмов? В качестве ответа запишите идентификатор этого пользователя.
ratings_movies.groupby('userId')['genres'].nunique().sort_values(ascending=False)

# Задание 5.
# Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
# В качестве ответа укажите идентификатор этого пользователя.
# Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), 
# можно воспользоваться методом agg() на сгруппированных данных.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


userId
599    524
414    482
448    403
380    399
474    395
      ... 
578     15
12      15
85      13
214     13
245     13
Name: genres, Length: 610, dtype: int64